# Reusing language keywords

- how single language keyword is used in different context; but keyword's semantic remained same
-  where it makes sense
- Examples
    - using
    - where
    - partial classes and methods
    - => lambdas and one liner method bodies and value tuples

In [ ]:
class Person
{
    string name;
    int age;

    public Person(string name, int age) =>
        (this.name, this.age) = (name, age);
}

# Seperating Responsibility 🤝

<img src=images/mediator.png width=600>

# Discussion: Method Parameters 🪂

Passing Parameter By Reference and how we start writing confusing code

In [ ]:
class Model { } // imagine Model has complex structure; lot of fields; nested model/collections

void UpdateSection1(Model model) { }
void UpdateSection2(Model model) { }

Data Access Layer
void UpdateFields(string firstName, string lastName, 1, 2, 3);




var model = new Model();
UpdateSection1(model);      // you dont see what fields are getting affected; you have to step into the code to understand it
UpdateSection2(model);      // you dont see if all necessary fields are appropriately set by first step so it can get consumed/used in step 2
                            // missing null checks
                            // some step is missed

In [ ]:
class Model { }

var model = new Model();

// do first things with model
if (true /*lot of checks to ensure first things were successful*/)
{
    // do second things with model
    if (true /*lot of checks to ensure second things were successful*/)
    {
        // do third things with model
        if (true /*lot of checks to ensure third things were successful*/)
        {
            // and so on....
        }
        //else how to recover out of this; undo second....undo first; things are getting out of control
    }
    //else how to recover out of this; undo first things?
}
//else how to recover out of this; log failure ofcourse; report to user? anything else?

In [ ]:
// we can make things linear

class Model { } // imagine Model has complex structure; lot of fields; nested model/collections

bool MultiStepMethod(Model model) // there are design patterns for this; but lets not go there now
{
    // do first things with model
    if (!true /*lot of checks to ensure first things were successful*/)
        // do we need to recover + undo previous steps?
        // log + report to user? anything else?
        return false; // or its an exception?

    // use return / exceptions
    // dont use exceptions as return mechanism

    // do second things with model
    if (!true /*lot of checks to ensure first things were successful*/)
        // do we need to recover + undo previous steps?
        // log + report to user? anything else?
        return false; // or its an exception?

    // do third things with model
    if (!true /*lot of checks to ensure first things were successful*/)
        // do we need to recover + undo previous steps?
        // log + report to user? anything else?
        return false; // or its an exception?

    // we are here and nothing wrong happened
    return true; /* finally */
}

MultiStepMethod(new Model());

- Some ways to get out of it

In [ ]:
class Model { }

Model UpdateSection1(Model model) => model; // for the sake of implementation
Model UpdateSection2(Model model) => model;

var model = new Model();
model = UpdateSection1(model);
// check model + log failures + recover etc
model = UpdateSection2(model);
// check model + log failures + recover etc

// we can wrap calls; but it can be confusing some times
var model2 = UpdateSection2(UpdateSection1(new Model())); // but how to do checks + failures?

In [ ]:
class Model { }

Model UpdateSection1(Model model)
{
    // do validations; which we should always
    bool validated = true;
    if (!validated) throw new ArgumentException();

    // do whatever
    try
    {
        return model;
    }
    catch (Exception ex)
    {
        //log + report + recover
        throw;
    }
}

Model UpdateSection2(Model model) => model; // same like above; but for sake of moving on

try
{
    var model = UpdateSection2(UpdateSection1(new Model())); // if in chain anything wrong happen; we will get exception
}
catch (Exception ex)
{
    // how to tell which step failed? we can have different wrapped exception; Step1Exception; Step2Exception and associated catch blocks
}

In [ ]:
class Model { }

static Model UpdateSection1(this Model model) => model;
static Model UpdateSection2(this Model model) => model;

// extension methods; we can chain the extension methods
var model = new Model()
    .UpdateSection1()
    .UpdateSection2();

In [ ]:
using static System.Console;

class Section1 { }
class Section2 : Section1 { }
class Model : Section2 { }

void UpdateSection(Section1 model) { WriteLine("Updating Section 1"); } // method overloads; less chance of missing step
void UpdateSection(Section2 model) { WriteLine("Updating Section 2"); } // method overloads

var model = new Model();

UpdateSection(model);               // will match to most closest overload; Section 2

UpdateSection(model as Section1);
UpdateSection(model as Section2);